In [1]:
import calendar
import datetime
import numpy as np
import os
import pandas as pd
import urllib.request
from urllib.request import urlopen
import plotly.express as px
import json
import requests
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import seaborn as sns
import calplot
import nltk
import folium
from folium import plugins
from folium.plugins import HeatMap
from scipy import stats # For in-built method to get PCC
from scipy.stats import pearsonr
from urllib.parse import quote
from matplotlib import rc
import warnings
warnings.filterwarnings('ignore')
from matplotlib import pyplot, dates
import bokeh 
from bokeh.io import push_notebook, show, output_notebook 
from bokeh.layouts import row
from bokeh.plotting import figure 
from bokeh.models import ColumnDataSource
output_notebook()
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
#
#plt.rcParams.update(plt.rcParamsDefault)
#plt.rcParams.update({'figure.figsize': (15, 5)})

Loading BokehJS ...


Shakes and Cracks below the surface: Is the impact the same or different? 

Main concept:

Exploring the inequalities and differences in reporting of Earthquakes occuring in different parts of the world/geographical regions. 
The social, political, and geographical impacts on regions after earthquakes.
Impact on a country’s GDP and financial growth.

More ideas
Despite tectonic plates existing in all parts of the world, there is a strong impact on some countries/regions compared to others.
Regions with the most number of occurrences.
10 highest magnitude earthquakes
Earthquake related to monsoon season, more infrastructural damage?
Time of day, weeks, months, years with most earthquakes, 


In [2]:
from pandas import read_excel

# Loading earthuake dataset 
df_EQ = pd.read_csv("Significant_Earthquake_Dataset_1900-2023.csv")
print(df_EQ.info)

# Loading GDP dataset 
GDP_all = pd.read_csv("gdp_1960_2020.csv")

# Loading Population dataset
Pop = pd.read_csv("world_population.csv")



<bound method DataFrame.info of                            Time                               Place  Latitude  \
0      2023-02-17T09:37:34.868Z        130 km SW of Tual, Indonesia   -6.5986   
1      2023-02-16T05:37:05.138Z       7 km SW of Port-Olry, Vanuatu  -15.0912   
2      2023-02-15T18:10:10.060Z         Masbate region, Philippines   12.3238   
3      2023-02-15T06:38:09.034Z     54 km WNW of Otaki, New Zealand  -40.5465   
4      2023-02-14T13:16:51.072Z         2 km NW of Lele?ti, Romania   45.1126   
...                         ...                                 ...       ...   
37326  1902-01-01T05:20:30.000Z      113 km ESE of Nikolski, Alaska   52.3800   
37327  1901-12-31T09:02:30.000Z       221 km SW of Nikolski, Alaska   51.4500   
37328  1901-12-30T22:34:00.000Z                     south of Alaska   52.0000   
37329  1901-03-03T07:45:00.000Z  12 km NNW of Parkfield, California   36.0000   
37330  1900-10-09T12:25:00.000Z      16 km SW of Old Harbor, Alaska   57.0900

The  most normal distributed columns are; magnitude, gap, dmin, rms, horizontal error, depth error, magnitude error.

The other appears to have a skewed distribution: Latitude, longitude, depth, nst, magnst.



In [4]:
# We take out a subset of the data we want to look into
df_EQ= df_EQ[df_EQ['Type'].isin(['earthquake'])]
df_EQ_cut = df_EQ[['Time','Place', 'Latitude', 'Longitude', 'Mag', 'Depth']]
df_EQ_cut.tail()

,Time,Place,Latitude,Longitude,Mag,Depth
37326,1902-01-01T05:20:30.000Z,"113 km ESE of Nikolski, Alaska",52.38,-167.45,7.00,NaN
37327,1901-12-31T09:02:30.000Z,"221 km SW of Nikolski, Alaska",51.45,-171.02,7.10,NaN
37328,1901-12-30T22:34:00.000Z,south of Alaska,52.00,-160.00,7.00,NaN
37329,1901-03-03T07:45:00.000Z,"12 km NNW of Parkfield, California",36.00,-120.50,6.40,NaN
37330,1900-10-09T12:25:00.000Z,"16 km SW of Old Harbor, Alaska",57.09,-153.48,7.86,NaN


In [5]:
# We convert out time column
# We filter the dataset from 2010 - 2023
df_EQ_cut['Time'] = pd.to_datetime(df_EQ_cut['Time'])
df_EQ_cut['year'] = df_EQ_cut['Time'].dt.year
df_filter = df_EQ_cut[df_EQ_cut['Time'].dt.year.between(2010, 2023)]


In [7]:
# Creating a column that specify country, based of the information in 'place' 
df_filter['Place'] = df_filter['Place'].astype(str)

for index, row in df_filter.iterrows():
    df_filter.loc[index, 'Country'] = row['Place'].split()[-1]

col = list(df_filter.columns)
col.insert(2, col.pop(col.index('Country')))
EQ_filter = df_filter[col]

#japan_data = EQ_filter[EQ_filter['Country'] == 'Japan']
#print(japan_data)

In [8]:
# Then we drop place as it is no longer needed
EQ_filter.drop(columns=["Place"], inplace=True)

In [9]:
EQ_filter

,Time,Country,Latitude,Longitude,Mag,Depth,year
0,2023-02-17 09:37:34.868000+00:00,Indonesia,-6.5986,132.0763,6.1,38.615,2023
1,2023-02-16 05:37:05.138000+00:00,Vanuatu,-15.0912,167.0294,5.6,36.029,2023
2,2023-02-15 18:10:10.060000+00:00,Philippines,12.3238,123.8662,6.1,20.088,2023
3,2023-02-15 06:38:09.034000+00:00,Zealand,-40.5465,174.5709,5.7,74.320,2023
4,2023-02-14 13:16:51.072000+00:00,Romania,45.1126,23.1781,5.6,10.000,2023
...,...,...,...,...,...,...,...
6366,2010-01-03 22:36:25.640000+00:00,Islands,-8.7830,157.3540,7.1,10.000,2010
6367,2010-01-03 21:48:02.870000+00:00,Islands,-8.7260,157.4870,6.6,10.000,2010
6368,2010-01-03 20:39:12.980000+00:00,Peru,-8.8020,-77.7180,5.7,116.800,2010
6369,2010-01-03 04:11:39.490000+00:00,Guinea,-5.2970,145.8590,5.8,34.000,2010


# Impact on a country’s GDP and financial growth. (Mabel)

In [10]:
# 
# Making coordinates pair for longitude and langitue 
EQ_filter['Coordinates'] = list(zip(EQ_filter.Longitude, EQ_filter.Latitude))


import plotly.express as px

fig = px.scatter_geo(EQ_filter, lat="Latitude", lon="Longitude", color="Mag",
                     hover_name="Country", size="Mag",animation_frame= "year",
                     projection="natural earth", size_max=8)

# Show figure 
fig.show()


Removal of rows with country names, such as "Ridge", "region", "nan", "Rise" as they are designated names for the earthquakes that appear in the oceans as seen on the map or mid-ocean ridge earthquakes. Even though the aftermath of these earthquakes may have impacted some countries. The datavisulisation would be focused on the earthquakes designated as to occuring in their country. 


In [11]:
# Removal of of the rows, designated under the country names "Ridge", "Region", "nan", "Rise"
EQ_filter = EQ_filter[~EQ_filter['Country'].isin(['Ridge', 'region','Rise','nan'])]

In [40]:
# New dataframe with magnitude above six
# Based of magnitude scale taken from an post by Michigan tech: 
# https://www.mtu.edu/geo/community/seismology/learn/earthquake-measure/magnitude/. 
EQ_filter_mag = EQ_filter[EQ_filter['Mag'] > 6.0]

In [30]:

# The 20 countries with highest occurence of earthquakes from 2010-2022
contry_count = EQ_filter.groupby('Country').size()
contry_count = contry_count.sort_values(ascending=False)
contry_20 =  contry_count.iloc[:20]

# The 20 countries with the highest occurence of earthquakes above the magnitude 6 from 2010-2022
contry_count_mag = EQ_filter_mag.groupby('Country').size()
contry_count_mag  = contry_count_mag.sort_values(ascending=False)
contry_20_mag  =  contry_count_mag.iloc[:20]

# Defining figure 
fig = go.Figure()

# Add trace for general earthquakes 
fig.add_trace(go.Bar(
    x=contry_20.index,
    y=contry_20.values,
    name='Countries',
    marker_color='powderblue'
))


# Add layout 
fig.update_layout(
    title='Top 20 countries with highest occurences of earthquakes (2010-2023)',
    xaxis_title='Country',
    yaxis_title='No. of earthquakes',
    barmode='group'
)


# Define subfigure 
fig2 = go.Figure()


# Add trace for earthquakes Mag>6.0
fig2.add_trace(go.Bar(
    x=contry_20_mag.index,
    y=contry_20_mag.values,
    name='Countries',
    marker_color='mediumpurple'
))

# Combine both plots using a subplot

fig3 = make_subplots(rows=2, cols=1, subplot_titles=('Top 20 Earthquake Frequency (2010-2023)', 'Top 20 Earthquakes Frequency > 6.0 (2010-2023)'))
fig3.add_trace(fig.data[0], row=1, col=1)
fig3.add_trace(fig2.data[0], row=2, col=1)

# Adjusting axes 
fig3.update_xaxes(title_text="Country", row=2, col=1)
fig3.update_yaxes(title_text="No. of earthquakes", row=1, col=1, title_standoff=10,title_font=dict(size=10))
fig3.update_yaxes(title_text="No. of earthquakes", row=2, col=1, title_standoff=10,title_font=dict(size=10))
fig3.update_layout(width=640,
    height=600,)
    
# Show figure    
fig3.show()

In [31]:
# Number of Earthquakes that occurred from 2010-2023 Japan vs Nepal 

# Filter for earthquakes in Japan and Nepal 
Japan = EQ_filter[(EQ_filter['Country'] == 'Japan') ]
Indo = EQ_filter[(EQ_filter['Country'] == 'Indonesia') ]

# We group the earthquakes by year and count occurences
japan_count = Japan.groupby('year').size()
Indo_count = Indo.groupby('year').size()

# Create a barchart showing the earthquake counts for each country
fig = go.Figure()

# Add trace for Indonesia count
fig.add_trace(go.Bar(
    x=japan_count.index,
    y=japan_count.values,
    name='Japan',
    marker_color='powderblue'
))

# Add trace for Indonesia count
fig.add_trace(go.Bar(
    x=Indo_count.index,
    y=Indo_count.values,
    name='Indonesia',
    marker_color='mediumpurple'
))

# Add layout 
fig.update_layout(
        title = go.layout.Title(
            text = 'Earthquake Frequency in Japan and Nepal (2010-2023)',
            x = 0.5),
    xaxis_title='Year',
    yaxis_title='No. of Earthquakes',
    width=640,
    height=600,
)

# Update logscale
fig.update_layout(yaxis_type='log')

# update show figure
fig.show()

In [15]:
# Number of Earthquakes that occurred from 2010-2023 Japan vs Nepal 
# Created stacked barchart showing the earthquake counts for each country

# Define figure
fig = go.Figure()


# Add traces 
fig.add_trace(go.Bar(
    x=japan_count.index,
    y=japan_count.values,
    name='Japan',
    marker_color='powderblue'
))

fig.add_trace(go.Bar(
    x=Indo_count.index,
    y=Indo_count.values,
    name='Indonesia',
    marker_color='mediumpurple'
))

# Add layout 
fig.update_layout(
        title = go.layout.Title(
            text = 'Earthquakes Occurences in Japan and Nepal (2010-2023)',
            x = 0.5),
    xaxis_title='Year',
    yaxis_title='No. of Earthquakes',
    barmode='stack'
)


fig.update_layout(yaxis_type='log')
fig.show()

In [16]:
# Extracting Japan and Indonesia
Japan_Indo = EQ_filter[(EQ_filter['Country'].isin(['Japan', 'Indonesia']))]
Japan_Indo

,Time,Country,Latitude,Longitude,Mag,Depth,year,Coordinates
0,2023-02-17 09:37:34.868000+00:00,Indonesia,-6.5986,132.0763,6.1,38.615,2023,"(132.0763, -6.5986)"
12,2023-02-07 00:35:51.673000+00:00,Indonesia,-7.3450,105.9037,5.5,57.855,2023,"(105.9037, -7.345)"
30,2023-01-26 04:23:51.234000+00:00,Indonesia,2.8388,126.9805,5.7,35.000,2023,"(126.9805, 2.8388)"
34,2023-01-24 02:13:14.926000+00:00,Indonesia,2.8209,126.9485,5.8,35.000,2023,"(126.9485, 2.8209)"
41,2023-01-18 08:01:26.318000+00:00,Indonesia,2.7047,127.1169,5.5,48.234,2023,"(127.1169, 2.7047)"
...,...,...,...,...,...,...,...,...
6323,2010-01-25 07:15:08.980000+00:00,Japan,30.9640,130.8880,5.5,38.200,2010,"(130.888, 30.964)"
6338,2010-01-17 06:04:36.520000+00:00,Japan,37.9380,143.5990,5.6,7.000,2010,"(143.599, 37.938)"
6341,2010-01-15 11:08:38.370000+00:00,Japan,26.7460,126.2850,5.7,139.100,2010,"(126.285, 26.746)"
6345,2010-01-13 17:18:07.990000+00:00,Indonesia,-0.6870,133.3300,5.8,10.000,2010,"(133.33, -0.687)"


A visualisation of the relationship between the magnitude and depth was created. 
To identify time and place for the most damaging earthquakes, based on theory for magnitude and depth in earthquakes.
Higher magnitude cause more damage, but in terms of the focal depth, shallow earthquakes tend to be more damaging than the deeper ones. The shallow earthquakes represent the ones occuring less than 50 kilometeres deep. 
https://quantectum.com/facts-about-earthquakes-depth/

In [38]:
# For more in depth understanding of the earthquakes.
# For this visualisation the Plotly.express documentation, was used as inspiration.

import plotly.graph_objects as go

# Create separate traces for each country
trace_Japan = go.Scatter(
    x=Japan_Indo[Japan_Indo['Country'] == 'Japan']['Depth'],
    y=Japan_Indo[Japan_Indo['Country'] == 'Japan']['Mag'],
    mode='markers',
    name='Japan',
    marker=dict(size=10, color='powderblue')
)

trace_Indo = go.Scatter(
    x=Japan_Indo[Japan_Indo['Country'] == 'Indonesia']['Depth'],
    y=Japan_Indo[Japan_Indo['Country'] == 'Indonesia']['Mag'],
    mode='markers',
    name='Indonesia',
    marker=dict(size=10, color='mediumpurple')
)

# Creates figure and add traces
fig = go.Figure(data=[trace_Japan, trace_Indo])

# Add animation frames based on the years
frames = []
years = Japan_Indo['year'].unique()
for year in years:
    frame_data = Japan_Indo[Japan_Indo['year'] == year]

    # Add scatterplot for Japan between Mag and depth for concerning year
    frame_japan = go.Scatter(
        x=frame_data[frame_data['Country'] == 'Japan']['Depth'],
        y=frame_data[frame_data['Country'] == 'Japan']['Mag'],
        mode='markers',
        name='Japan',
        marker=dict(size=10, color='powderblue')
    )

     # Add scatterplot for Japan between Mag and depth for concerning year
    frame_Indo = go.Scatter(
        x=frame_data[frame_data['Country'] == 'Indonesia']['Depth'],
        y=frame_data[frame_data['Country'] == 'Indonesia']['Mag'],
        mode='markers',
        name='Indonesia',
        marker=dict(size=10, color='mediumpurple')
    )
    frames.append(go.Frame(data=[frame_japan, frame_Indo]))

# Add the frames to figure
fig.frames = frames

# Updates the layout
fig.update_layout(
        title = go.layout.Title(
            text = 'Magnitude against depth in earthquakes for Japan and Nepal (2010-2023)',
            x = 0.5),   
    xaxis=dict(
        title='Depth (km)',
        gridcolor='white',
        type='log',
        gridwidth=2,
    ),
    yaxis=dict(
        title='Magnitude',
        gridcolor='white',
        gridwidth=1,
        fixedrange=False,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    width=800,
    height=600
)

# Adding animation buttons
fig.update_layout(
    updatemenus=[
        dict(
            type='buttons',
            buttons=[
                dict(label='play',
                     method='animate',
                     args=[None, {'frame': {'duration': 800, 'redraw': True}, 'fromcurrent': True, 'transition': {'duration': 0}}]),
                dict(label='Pause',
                     method='animate',
                     args=[[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate', 'transition': {'duration': 0}}])
            ],
            showactive=True,
            x=0.05,
            y=0,
            xanchor='left',
            yanchor='bottom'
        ),
    ]
)

# Show figure
fig.show()


In [37]:
# Visualisation of GDP from 1960 till 2020 using linechart
fig = px.line(GDP_Japan_Indo, x="year", y="gdp", color="country",color_discrete_map={"Japan": "blue", "Indonesia": "purple"})
fig.update_traces(textposition="bottom right")
fig.update_layout(title= go.layout.Title(text = 'The Economic Growth for Earthquake-Prone Countries',x=0.456,y=0.95),
    xaxis_title='Year',
    yaxis_title='GDP(USD)',
    width=640,
    height=600,)
fig.show()

GDP dataset: 

This was downloaded from https://www.kaggle.com/datasets/holoong9291/gdp-of-all-countries19602020/code. It contains GDP data of all countries and entities from 1960-2020. self-updating dataset and contains 5 columns: title, pubDate, guid, link and description. It includes 16193 unique values which are then filtered. 


Datavisulisation:

Plotly.express linechart. This is used to support the history given in the story and support the understanding of the financial growth narrated. Also help give insight into pre-financial conditions as the story also briefly touches. Additionally, by having simple interactivity such as displaying year and gpd, the reader is enables to follow the story and identify periods of rapid growth as the story describes. 
As well as other patterns like, steady consistent growth. 
This aapproach enhances the narrative and helps keep the reader engaged as well. 



6. **Discussion.** Think critically about your creation

- What went well?
D

The research 

- What is still missing? What could be improved?, Why?

For the scope of the project, since the focus was on the social aspect - analysing the reporting differences for earthquake news in media - the initial dataset was inadequate since it only included (detailed) data on earthquake instances. Additional datasets and research was then done to get a deeper understanding of various types of reporting and its possible consequences. 

The project scope was limited to the recent two decades, with some segments of the article only including data between 2010 and 2023. Additionally, the data from 2023 is limited to March, which can be considered incomplete data, but was included due to the Feburary 2023 earthquakes in Turkey and Syria. While necessary for the scope of the project, the limited range of data provided a limited range of comparison. For further analysis, the timeline could be expanded to include historical data, and along with this would follow a more comprehensive scope for the number of countries included in the analysis. A metric for selection of countries for a different kind of comparison could also have been a larger grographical, financial or cultural difference, which may impact style of reporting (which may also differ due to cultural or social aspects). 

Delivering a story that aims to state and visualise the econimical impacts post-earthquakes came with great concerns. As it quickly became clear that the original dataset was not adequate enough to help visualise the focus angle.
Eventhough, the earthquake data was supported by World GDP data. This data only represented some part of the impact. Yes, it generally tells if a country is performing well and the size of an economy. However, in order to clearly dissect the impact, there was a neccessity for data describing broader areas of the different sectors. Infrastrutural data, data about damaged property, reconstruction fees and poverty rate etc. Perhaps even stocks of leading firms, that generate most of the countries' wealth. 

The lack of more datavisulisations, made the 'economic impact' section heavy in terms of reading, but also less engaging than what it could have been.
Another point to takeway, was the difficulties in using the Earthquake data and GDP together in a sensible way. First the attention was to look at the yearly change in GDP after a high magnitude earthquake and to use trend analysis to look for a significant drop or just look at the GPD yearly alone. Then compare the results side by side for each country.
However, it would be hard to measure and compare it against the countries. As it would require to identify earthquakes that impacted similarly in terms of magnitude, area, loss of life. 
Furthermore, it should be around the same time to account for updated technology etc.
However, one way around it could be through averaging the measure found, for the economical impact after earthquake strikes. Overall, without the accompanying article it would have been hard to explain the economical differences between the two countries. In addition, the narrative story became centered around pre-economic conditions and the difference in prevention methods. 
It is reasonable to assume, that there could have been more in depth exploration of the economical impact. Additionally, more visualisation that not only address statistical factors but also present self analysed result, such as predictions for future impacts and so on.

Natural Language Processing (NLP) in the form of word clouds was used in order to visually represent some of the wording choices in news headlines and descriptions. The dataset used for the word cloud is from BBC News, and other news publications may operate in a slightly differently manner. For the wordcloud, the python package was used, which meant that many of the parameters were in-built. For future work, a dataset that includes historical data could be used for a more encompassing comparison and analysis, and a different method could be used to create the visualisation. 

For the purpose of future work, additional datasets with some form of social data relating to reporting methods or web scraping of news articles could be amongst the methods used for a deeper analysis. Additionally, methods within statistical analsyis, Text mining and Sentiment Analysis machine learning (ML) and deep learning (DL) such as Natural Language Processing (NLP) [https://www.ibm.com/topics/natural-language-processing#:~:text=NLP%20combines%20computational%20linguistics—rule,learning%2C%20and%20deep%20learning%20models.] could be used to discover reporting trends.